In [ ]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam, SGD, RMSprop

import tensorflow as tf

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def DistanceMatrix_cpu(boundary_x, boundary_y, internal_points_x, internal_points_y):
    dist = []
    dist_x = (boundary_x[:,npy.newaxis] - internal_points_x[npy.newaxis,:])**2

    dist_y = (boundary_x[:,npy.newaxis] - internal_points_y[npy.newaxis,:])**2

    return np.sqrt(dist_x+dist_y)


In [ ]:
from numpy import inf
import numpy as npy
from timeit import default_timer as timer

def gen_circle(img):

        Boundary_Points = 1000

        # y_in, x_in = npy.where(img[:,:,0] == 220)
        
        y_in, x_in = npy.where(img != 0)
        print(y_in.shape)
        if y_in.shape[0] == 0:
          return None

        Circ_Bound = np.linspace(0, 2*np.pi, Boundary_Points); 
        candidate_circle = 0
        highest_size = 0
 
        print('found candidate circle')

        vv = npy.size(y_in);
        ww = npy.size(x_in);
        zz = npy.zeros(ww);
        
        #Create circle boundary
        R = img.shape[0] // 2
        x = int(img.shape[0] // 2)
        y = int(img.shape[1] // 2)

        #Find all points within circle by first getting indices of all points in the image
#         xq, yq = np.where(img != -1)
        y_in_circle, x_in_circle = y_in, x_in #npy.where(img[:,:,2] != -1)


        normalized = np.asarray(npy.column_stack([x_in_circle, y_in_circle]))#NormalizeData(npy.column_stack([x_in_circle, y_in_circle])) * 10.
    
        Circ_Bound_x = R * np.cos(Circ_Bound) + x
        Circ_Bound_y = R * np.sin(Circ_Bound) + y    

        DM_data_Sample = DistanceMatrix_cpu((Circ_Bound_x), (Circ_Bound_y), normalized[:,0], normalized[:,1]);


        IM = 1./DM_data_Sample; 
        Boundary_Values_Sample = np.sum(IM, axis=1);
        Boundary_Values_Normalized = Boundary_Values_Sample

        return Boundary_Values_Normalized

In [ ]:
import cv2 


dataset = {
    'normal' : [],
    'covid' : [],
}

for img in normal_images:


    scale_percent = 60 # percent of original size
    width = 700#int(img.shape[1] * scale_percent / 100)
    height = 700#int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    b = gen_circle(img)
    dataset['normal'].append(b)
    del b


In [ ]:
for img in covid_images:
#     img = normal_images[0]

    img = rgb2gray(img)
    scale_percent = 60 # percent of original size
    width = 700#int(img.shape[1] * scale_percent / 100)
    height = 700#int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    b = gen_circle(img)
    
    dataset['covid'].append(b)
    del b

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])



In [ ]:
train_x = []
test_x = []
train_y=[]
test_y=[]

i = 0
for shape in dataset:
    data = dataset[shape]

    if len(data) == 0:
        continue
        
    print('shape', shape)

    data = np.asarray(data) 
    # data = data[:,0]
    data = np.abs(np.apply_along_axis(np.fft.fft, 1, data))
    random_range = np.arange(data.shape[0])
    np.random.shuffle(random_range)
    train_range = int(random_range.shape[0] *.7)
    
    if i== 0:
        train_y = [i] * data[random_range[:train_range]].shape[0]
        test_y = [i] * data[random_range[train_range:]].shape[0]
        train_x = data[random_range[:train_range]]
        test_x = data[random_range[train_range:]]
    else:
        train_y = np.concatenate((train_y, [i] * data[random_range[:train_range]].shape[0]), axis=0)
        test_y = np.concatenate((test_y, ([i] * data[random_range[train_range:]].shape[0])), axis=0)

        train_x = np.concatenate((train_x, data[random_range[:train_range]]), axis=0)

        test_x = np.concatenate((test_x, data[random_range[train_range:]]), axis=0)
    i+=1

In [ ]:
train_x.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(np.nan_to_num(train_x), train_y)
pred_i = neigh.predict(np.nan_to_num(test_x))
neigh.score(np.nan_to_num(test_x), test_y)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import svm

clf = svm.SVC(decision_function_shape='ovo', probability=True)
clf.fit(np.nan_to_num(train_x), train_y)

predicted = clf.predict(np.nan_to_num(test_x))

# get the accuracy
accuracy_score(test_y, predicted)

In [ ]:
DATASET_DIR = "../input/covid-19-x-ray-10000-images/dataset"

In [ ]:
os.listdir(DATASET_DIR)


In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

normal_images = []
for img_path in glob.glob(DATASET_DIR + '/normal/*'):
    normal_images.append(mpimg.imread(img_path))

fig = plt.figure()
fig.suptitle('normal')
plt.imshow(normal_images[10], cmap='gray') 

covid_images = []
for img_path in glob.glob(DATASET_DIR + '/covid/*'):
    covid_images.append(mpimg.imread(img_path))

fig = plt.figure()
fig.suptitle('covid')
plt.imshow(covid_images[10], cmap='gray') 

In [ ]:
print(len(normal_images))
print(len(covid_images))

In [ ]:
IMG_W = 150
IMG_H = 150
CHANNELS = 3

INPUT_SHAPE = (IMG_W, IMG_H, CHANNELS)
NB_CLASSES = 2
EPOCHS = 48
BATCH_SIZE = 6

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(Conv2D(250,(3,3)))
model.add(Activation("relu"))
  
model.add(Conv2D(128,(3,3)))
model.add(Activation("relu"))
model.add(AvgPool2D(2,2))
model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(AvgPool2D(2,2))

model.add(Conv2D(256,(2,2)))
model.add(Activation("relu"))
model.add(MaxPool2D(2,2))
    
model.add(Flatten())
model.add(Dense(32))
model.add(Dropout(0.25))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    DATASET_DIR, 
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= False,
    subset='validation')

history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print("training_accuracy", history.history['accuracy'][-1])
print("validation_accuracy", history.history['val_accuracy'][-1])

In [ ]:
label = validation_generator.classes

In [ ]:
pred= model.predict(validation_generator)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (validation_generator.class_indices)
labels2 = dict((v,k) for k,v in labels.items())
predictions = [labels2[k] for k in predicted_class_indices]
print(predicted_class_indices)
print (labels)
print (predictions)

In [ ]:
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(predicted_class_indices,label)
cf

In [ ]:
exp_series = pd.Series(label)
pred_series = pd.Series(predicted_class_indices)
pd.crosstab(exp_series, pred_series, rownames=['Actual'], colnames=['Predicted'],margins=True)

In [ ]:
plt.matshow(cf)
plt.title('Confusion Matrix Plot')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show();